# Introduction

The [TinyLlama](https://github.com/jzhang38/TinyLlama) project "aims to pretrain a 1.1B Llama model on 3 trillion tokens." 1.1B tokens represents a considerable step up from the small GPT model we [previously fine-tuned](../2_gpt2_single_gpu/2.%20GPT2%20on%20a%20single%20GPU.ipynb). That model had 124M parameters; TinyLlama, while still small by the standards of most widely-used LLMs, is almost ten times the size. We will need around 20GB VRAM at a bare minimum to fine-tune this model.

## Instruction Tuning
We are going to focus on instruction tuning in this example. Instruction Tuning is a supervised learning technique in which we train the model on instruction/output pairs with the goal of training the model to follow human instructions. Before instruction tuning, the base model is trained on next-token completion. We saw this in the GPT2 example: we provided the start of a story and the model completed it. An instruction-tuned model, on the other hand, is trained to answer a question or instruction.

[This repository](https://github.com/xiaoya-li/Instruction-Tuning-Survey) contains a wealth of information on the current state of the field of instruction tuning.

The [TinyLlama repository](https://github.com/jzhang38/TinyLlama/tree/main/sft) includes scripts for fine-tuning. While these will be useful references, we will try to proceed with an approach similar to that used in the gpt2 and t5-small notebooks--purely for the sake of making this notebook a reasonable learning step following those.

# The Data
We will use the [SlimOrca](https://huggingface.co/datasets/Open-Orca/SlimOrca) dataset. This is a curated subset of the much larger [OpenOrca](https://huggingface.co/datasets/Open-Orca/OpenOrca) dataset. Why this dataset? It's one of the most popular sources of instruction data on Hugging Face, and its size is more manageable than the full OpenOrca dataset. That's all!

# 1. Load the model and try some examples

We'll begin, as always, by loading the model and trying out some examples.

In [11]:
# Some Environment Setup
OUTPUT_DIR = "../results/TinyLlama/" # the path to the output directory; where model checkpoints will be saved
LOG_DIR = "../logs/TinyLlama/" # the path to the log directory; where logs will be saved
CACHE_DIR = "../cache/TinyLlama/" # the path to the cache directory; where cache files will be saved

In [8]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

model_ckpt = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"

tokenizer = AutoTokenizer.from_pretrained(
    model_ckpt,
)

model = AutoModelForCausalLM.from_pretrained(
    model_ckpt,
    device_map="auto",
)

# Inference
def generate(prompt, max_new_tokens=100):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
    gen_tokens = model.generate(input_ids, max_new_tokens=max_new_tokens,
                                eos_token_id=tokenizer.eos_token_id,
                                repetition_penalty=1.1)
    return tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)[0]

print(generate("Here are step-by-step instructions to make a great cup of coffee with a Chemex coffee maker:\n1."))

Here are step-by-step instructions to make a great cup of coffee with a Chemex coffee maker:
1. Prepare the grounds
2. Add water
3. Grind the beans
4. Pour the water into the pot
5. Let it brew
6. Enjoy your coffee!
What is a Chemex?
A Chemex is a type of coffee maker that uses a ceramic filter to extract the flavor from the coffee beans. The Chemex was invented by a man named John C. Hunt in 1908


In this example, we structured our prompt with completion in mind: we generated the first part of the full text and asked the model to complete it. What happens if, instead, we ask a question or give an instruction?

In [9]:
# Question
print(generate("How do I make coffee with a Chemex coffee maker?"))

How do I make coffee with a Chemex coffee maker?
How do you make coffee with a Chemex?
What is the best way to brew coffee with a Chemex?
How do you make coffee with a Chemex filter?
How do you make coffee with a Chemex filter and a Chemex?
How do you make coffee with a Chemex filter and a Chemex filter?
How do you make coffee with a Chemex filter and a Chemex filter?
How do you make coffee with a Chemex filter and


In [10]:
# Instruction
print(generate("Tell me how to make coffee with a Chemex coffee maker."))

Tell me how to make coffee with a Chemex coffee maker.
I'm not sure if you can get the Chemex in the UK, but I think it's worth a try.
The Chemex is a great coffee maker and I love mine!
I have one of these and I love it. It makes a great cup of coffee every time.
I have a Chemex and I love it. It makes a great cup of coffee every time.
I have a Chemex and I love it. It makes a great cup


These did not work because the model has not been instruction tuned. Our task is to change that!

# 2. Getting and Exploring the Data

In [15]:
from datasets import load_dataset
from pathlib import Path

slimorca = load_dataset('Open-Orca/SlimOrca',
                           cache_dir=str(Path(CACHE_DIR) / "data"))


In [16]:
import json
print(json.dumps(slimorca["train"][0], indent=4))

{
    "conversations": [
        {
            "from": "system",
            "value": "You are an AI assistant. You will be given a task. You must generate a detailed and long answer.",
            "weight": null
        },
        {
            "from": "human",
            "value": "Write an article based on this \"A man has been charged with murder and attempted murder after a woman and the man she was on a date with were stabbed at a restaurant in Sydney, Australia.\"",
            "weight": 0.0
        },
        {
            "from": "gpt",
            "value": "Title: Tragedy Strikes in Sydney: Victims Stabbed during Dinner Date, Suspect Charged with Murder and Attempted Murder\n\nIntroduction\n\nA seemingly ordinary evening took a tragic turn in Sydney, Australia when a romantic dinner date at a local restaurant ended in bloodshed, as a man allegedly attacked a couple, stabbing both victims. Local authorities have charged the suspect with murder and attempted murder in connectio

You'll see that there are three components to the sample entry:
1. A *system message*: this should be familiar if you've used e.g. ChatGPT via the API. This is a general instruction specifying the model's role/identity and general instructions.
2. A *human message*: this is the specific instruction passed to the model by a human.
3. a *gpt*: this is the AI model's response.

So we want to use this dataset to fine-tune the model such that it will respond more like the *gpt* message when given the *system* and *human* messages.

First, we need to get these entries into a format we can actually use for fine-tuning.